In [1]:
from pathlib import Path

import pandas as pd

In [2]:
DATA_PATH = Path('./data/')
SAMPLES = ['train', 'val', 'test']
TRACKS = ['agnostic', 'aware']

# Load data

In [3]:
# load data
df = {}
for set_ in SAMPLES:
    df[set_] = {}
    for track in TRACKS:
        df[set_][track] = pd.read_json(DATA_PATH / set_  / (set_ + '.model-' + track + '.json'))
df_trial = pd.read_json(DATA_PATH / 'trial' / 'trial.json')

# prepocess data
df['train']['aware'].at[10519, 'src'] = 'None'
df['train']['agnostic'].drop(columns=['model'], inplace=True)
df['val']['agnostic'].drop(columns=['model'], inplace=True)

# Trial data

In [4]:
df_trial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hyp               80 non-null     object 
 1   ref               80 non-null     object 
 2   src               80 non-null     object 
 3   tgt               80 non-null     object 
 4   model             80 non-null     object 
 5   task              80 non-null     object 
 6   labels            80 non-null     object 
 7   label             80 non-null     object 
 8   p(Hallucination)  80 non-null     float64
dtypes: float64(1), object(8)
memory usage: 5.8+ KB


In [5]:
df_trial.head()

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,"A district of Kowloon, China.",tgt,The City <define> Chiuchow </define> is Kowloo...,The Chaoshan region where the Teochew dialect ...,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000
1,(Wicca) A witchdoctor.,tgt,"“ I went to see the witchdoctor , the <define>...",(South Africa) A traditional tribal diviner or...,,DM,"[Hallucination, Hallucination, Not Hallucination]",Hallucination,0.666667
2,(nautical) A halyard.,tgt,"Haul away , keeping strain on both parts of th...",(nautical) A staff that carries a flag or penn...,,DM,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.333333
3,The quality of being rational.,tgt,And what difference is there between the credi...,(archaic) The quality or state of being rational.,,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.000000
4,(uncountable) The study of trees.,tgt,It is now generally supposed that the forbidde...,The worship of trees.,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000


In [6]:
df_trial[['task', 'ref', 'model']].value_counts()

task  ref     model                      
DM    tgt                                    36
MT    either                                 35
PG    either  tuner007/pegasus_paraphrase     9
Name: count, dtype: int64

# Train data (agnostic)

In [7]:
df['train']['agnostic'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hyp     30000 non-null  object
 1   tgt     30000 non-null  object
 2   src     30000 non-null  object
 3   ref     30000 non-null  object
 4   task    30000 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


In [8]:
df['train']['agnostic'].head()

,hyp,tgt,src,ref,task
0,"Don't worry, it's only temporary.",Don't worry. It's only temporary.,Не волнуйся. Это только временно.,either,MT
1,Tom is never where he should be.,Tom is never where he's supposed to be.,"Тома никогда нет там, где он должен быть.",either,MT
2,It's hard for me to work with Tom.,I have trouble working with Tom.,Мне сложно работать с Томом.,either,MT
3,"Water, please.",I'd like some water.,"Воду, пожалуйста.",either,MT
4,I didn't expect Tom to betray me.,I didn't think that Tom would betray me.,"Я не ожидал, что Том предаст меня.",either,MT


In [9]:
df['train']['agnostic'][['task', 'ref']].value_counts()

task  ref   
DM    tgt       10000
MT    either    10000
PG    src       10000
Name: count, dtype: int64

# Train data (aware)

In [10]:
df['train']['aware'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hyp     30000 non-null  object
 1   tgt     30000 non-null  object
 2   src     30000 non-null  object
 3   ref     30000 non-null  object
 4   task    30000 non-null  object
 5   model   30000 non-null  object
dtypes: object(6)
memory usage: 1.4+ MB


In [11]:
df['train']['aware'].head()

,hyp,tgt,src,ref,task,model
0,Of or pertaining to the language of a particul...,"Of or pertaining to everyday language , as opp...","There are blacktips , silvertips , bronze whal...",tgt,DM,ltg/flan-t5-definition-en-base
1,Not coercive ; not involving coercion,Not coercive ; free of coercion,Mr. obama signed executive orders requiring al...,tgt,DM,ltg/flan-t5-definition-en-base
2,To express or express by words ; to express by...,To depict or portray .,Disloyal ? / the word is too good to paint out...,tgt,DM,ltg/flan-t5-definition-en-base
3,Having the power to authoritatively speak or w...,Having a commanding style .,"He instructed us in that booming , authoritati...",tgt,DM,ltg/flan-t5-definition-en-base
4,Without a scot .,"Without consequences or penalties , to go free...",To get off scot-free . What is the meaning of ...,tgt,DM,ltg/flan-t5-definition-en-base


In [12]:
df['train']['aware'][['task', 'ref', 'model']].value_counts()

task  ref     model                           
DM    tgt     ltg/flan-t5-definition-en-base      10000
MT    either  facebook/nllb-200-distilled-600M    10000
PG    src     tuner007/pegasus_paraphrase         10000
Name: count, dtype: int64

# Validation data (agnostic)

In [13]:
df['val']['agnostic'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hyp               499 non-null    object 
 1   ref               499 non-null    object 
 2   src               499 non-null    object 
 3   tgt               499 non-null    object 
 4   task              499 non-null    object 
 5   labels            499 non-null    object 
 6   label             499 non-null    object 
 7   p(Hallucination)  499 non-null    float64
dtypes: float64(1), object(7)
memory usage: 31.3+ KB


In [14]:
df['val']['agnostic'].head()

,hyp,ref,src,tgt,task,labels,label,p(Hallucination)
0,Resembling or characteristic of a weasel.,tgt,The writer had just entered into his eighteent...,Resembling a weasel (in appearance).,DM,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.2
1,Alternative form of sheath knife,tgt,Sailors ' and fishermen 's <define> sheath - k...,.,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,0.8
2,(obsolete) A short period of time.,tgt,"As to age , Bead could not form any clear impr...","(poetic) An instant, a short moment.",DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
3,(slang) An incel.,tgt,Because redpillers are usually normies or <def...,"(incel, _, slang) A man of a slightly lower ra...",DM,"[Not Hallucination, Not Hallucination, Halluci...",Not Hallucination,0.2
4,"An island in Lienchiang County, Taiwan.",tgt,On the second day of massive live - fire drill...,"An island in Dongyin, Lienchiang, Taiwan, in t...",DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0


In [15]:
df['val']['agnostic'][['task', 'ref']].value_counts()

task  ref   
DM    tgt       187
MT    either    187
PG    either    125
Name: count, dtype: int64

# Validation data (aware)

In [16]:
df['val']['aware'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hyp               501 non-null    object 
 1   ref               501 non-null    object 
 2   src               501 non-null    object 
 3   tgt               501 non-null    object 
 4   model             501 non-null    object 
 5   task              501 non-null    object 
 6   labels            501 non-null    object 
 7   label             501 non-null    object 
 8   p(Hallucination)  501 non-null    float64
dtypes: float64(1), object(8)
memory usage: 35.4+ KB


In [17]:
df['val']['aware'].head()

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,A sloping top .,tgt,The sides of the casket were covered with heav...,A decorative feature that sits on top of somet...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Not Halluci...",Hallucination,0.6
1,To react too much .,tgt,Please try not to overreact if she drives badl...,To react too much or too intensely .,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
2,The process of spoiling ; the state of being s...,tgt,"To prevent spoilage , store in a cool , dry pl...",The process of spoiling .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.6
3,To arrange in a particular way .,tgt,The way the opposition has framed the argument...,To construct in words so as to establish a con...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Not Halluci...",Hallucination,0.6
4,A feeling of concern ; a feeling of anxiety .,tgt,To mix with thy concernments i desist . What i...,That in which one is concerned or interested ;...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Hallucinati...",Hallucination,0.6


In [18]:
df['val']['aware'][['task', 'ref', 'model']].value_counts()

task  ref     model                           
DM    tgt     ltg/flan-t5-definition-en-base      188
MT    either  facebook/nllb-200-distilled-600M    188
PG    src     tuner007/pegasus_paraphrase         125
Name: count, dtype: int64

# Test data (agnostic)

In [19]:
df['test']['agnostic'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1500 non-null   int64  
 1   src               1500 non-null   object 
 2   tgt               1500 non-null   object 
 3   hyp               1500 non-null   object 
 4   task              1500 non-null   object 
 5   labels            1500 non-null   object 
 6   label             1500 non-null   object 
 7   p(Hallucination)  1500 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 93.9+ KB


In [20]:
df['test']['agnostic'].head()

,id,src,tgt,hyp,task,labels,label,p(Hallucination)
0,1,"Ты удивишься, если я скажу, что на самом деле ...",Would you be surprised if I told you my name i...,You're gonna be surprised if I say my real nam...,MT,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
1,2,Еды будет полно.,There will be plenty of food.,The food will be full.,MT,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.8
2,3,"Думаете, Том будет меня ждать?",Do you think that Tom will wait for me?,You think Tom's gonna wait for me?,MT,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.2
3,6,Два брата довольно разные.,The two brothers are pretty different.,There's a lot of friends.,MT,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,1.0
4,7,<define> Infradiaphragmatic </define> intra- a...,(medicine) Below the diaphragm.,(anatomy) Relating to the diaphragm.,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,0.8


In [21]:
df['test']['agnostic'][['task']].value_counts()

task
DM      563
MT      562
PG      375
Name: count, dtype: int64

# Test data (aware)

In [22]:
df['test']['aware'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1500 non-null   int64  
 1   src               1500 non-null   object 
 2   tgt               1500 non-null   object 
 3   hyp               1500 non-null   object 
 4   task              1500 non-null   object 
 5   labels            1500 non-null   object 
 6   label             1500 non-null   object 
 7   p(Hallucination)  1500 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 93.9+ KB


In [23]:
df['test']['aware'].head()

,id,src,tgt,hyp,task,labels,label,p(Hallucination)
0,0,"Sadly , at the end of his life , Luther became...",Crazy person ; crackpot or lunatic .,A crazy person .,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
1,4,इतिहासकार लोगन FBI के पिछला नीति के बहुत आलोचन...,Historians have criticized past FBI policies f...,Historians have strongly criticized the FBI's ...,MT,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
2,5,Àwọn erekùsù ìlà oòrùn adúláwọ̀ wà ní òkun Ind...,The East African Islands are in the Indian Oce...,The eastern islands of the Indian Ocean are lo...,MT,"[Not Hallucination, Hallucination, Hallucinati...",Hallucination,0.6
3,9,Smith was first introduced into society at the...,High society .,"The social group of a person , especially a wo...",DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,1.0
4,11,Ngini mamakai roket raksasa malabihi 100 kaki ...,It takes a giant rocket over a 100 feet high t...,It uses a giant rocket over 100 feet high to l...,MT,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.4


In [24]:
df['test']['aware'][['task']].value_counts()

task
MT      563
DM      562
PG      375
Name: count, dtype: int64

# Some check

In [25]:
i = -1

In [26]:
i += 1
step = 30
df['train']['agnostic'].loc[df['train']['agnostic']['task'] == 'MT', 'src'].iloc[step*i:step*(i+1)]

0                     Не волнуйся. Это только временно.
1             Тома никогда нет там, где он должен быть.
2                          Мне сложно работать с Томом.
3                                     Воду, пожалуйста.
4                    Я не ожидал, что Том предаст меня.
5                     Всё не так плохо, как думает Том.
6                                        Не усложняйте.
7     Почему вы не можете принять меня таким, какой ...
8                                   Ненавижу это место.
9             Том находится там в качестве добровольца.
10                                 Можно мне тоже одну?
11              Ты в жизни не такой, как на фотографии.
12          Она начала осваиваться в своем новом офисе.
13                               У меня сейчас нет дел.
14                Экватор делит землю на два полушария.
15                       Я хотел бы стать христианином.
16                           Все будут тебе завидовать.
17                        Том купил себе новое п